<a href="https://colab.research.google.com/github/Muralisekharbirlangi/Smart-City-assistant/blob/main/smartcity_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: I want to build a generative AI application called "Sustainable smart City assistant using IBM granite LLM" i want to integrate ibm-granite/granite-3.3-2b-instruct model from hugging class ,I want to deploy it with FastAPI so give me html and css/Gradio in google colab.My application has functionalities like:
# 1) Eco-Query Assistant: Here a user gives a sustainability-related question (eg, “How can I reduce my home energy usage?” or “Where is the nearest e-waste recycling center?”), and the model provides informative, location-sensitive, and practical responses.
# 2) Smart Complaint Resolver: Here a user enters a civic issue (eg, garbage overflow, pothole, water wastage), and the model classifies the issue, suggests possible causes, and routes it to the appropriate department (simulated or integrated with a mock API)

# The provided Gradio interface is a single input/output for a generic prompt.
# To implement the specific functionalities (Eco-Query and Smart Complaint Resolver),
# we need to modify the Gradio interface and the granite_inference function.

# We can use Gradio's Tabs or Radio buttons to let the user choose the functionality.
# The inference function will then need to process the input differently based on the chosen mode.

import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Install necessary libraries
# !pip install transformers accelerate gradio # Already installed by the user

# Define the model ID
model_id = "ibm-granite/granite-3.3-2b-instruct"

# Load the tokenizer and model
# This can be memory intensive. You might need a High-RAM runtime.
# Consider using lower precision (e.g., bfloat16 or float16 if supported) or quantization if needed.
try:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    # Use bfloat16 for potentially lower memory usage and faster inference on supported hardware
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")
    model.eval() # Set model to evaluation mode
except Exception as e:
    print(f"Error loading model: {e}")
    print("You might need a higher RAM or GPU runtime, or consider quantization.")
    # Provide a placeholder function if model loading fails
    def placeholder_inference(prompt, mode):
        return "Model loading failed. Cannot generate response."
    inference_function = placeholder_inference
    model = None # Indicate model loading failed
    tokenizer = None # Indicate tokenizer loading failed


# Define the function to generate text using the model with modality
if model and tokenizer:
    def granite_inference_with_mode(prompt, mode):
        try:
            # Prepare the prompt based on the selected mode
            if mode == "Eco-Query Assistant":
                formatted_prompt = f"Instruct: As a sustainable smart city assistant, answer this eco-sustainability question:\n{prompt}\nOutput:"
            elif mode == "Smart Complaint Resolver":
                # For complaint resolution, we might add instructions for classification and routing
                formatted_prompt = f"Instruct: As a smart city complaint resolver, analyze this civic issue. Classify the issue, suggest possible causes, and suggest which department should handle it (e.g., Waste Management, Public Works, Water Department, etc.).\nIssue: {prompt}\nOutput:"
            else:
                formatted_prompt = f"Instruct: {prompt}\nOutput:" # Default behavior

            inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

            # Generate response
            outputs = model.generate(
                inputs.input_ids,
                max_new_tokens=1000, # Adjust as needed
                num_return_sequences=1,
                temperature=0.7,    # Adjust for creativity vs. determinism
                top_p=0.9,          # Adjust for nucleus sampling
                do_sample=True,     # Enable sampling
                early_stopping=True
            )

            # Decode the output
            response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
            return response.strip()
        except Exception as e:
            return f"An error occurred during inference: {e}"

    inference_function = granite_inference_with_mode
else:
     # inference_function is already set to placeholder_inference if model load failed
     pass


# Create a Gradio interface with options for different functionalities
if inference_function: # Only create interface if we have a function to call
    with gr.Blocks(title="Sustainable Smart City Assistant") as demo:
        gr.Markdown(
            """
            # Sustainable Smart City Assistant powered by IBM Granite LLM
            Choose a functionality below and enter your query.
            """
        )

        with gr.Row():
            mode_choice = gr.Radio(
                ["Eco-Query Assistant", "Smart Complaint Resolver"],
                label="Choose Functionality",
                value="Eco-Query Assistant" # Default value
            )

        input_text = gr.Textbox(lines=5, label="Enter your query or issue:")
        output_text = gr.Textbox(label="Generated Output:", interactive=False)

        submit_button = gr.Button("Submit")

        # Link the button click to the inference function, passing both input and mode
        submit_button.click(
            fn=inference_function,
            inputs=[input_text, mode_choice],
            outputs=output_text
        )

    # Launch the Gradio interface
    # share=True will create a public link (expires after 72 hours)
    demo.launch(share=True)
else:
    print("Could not create Gradio interface because model loading failed.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5f703b18867ebdf263.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# New Section